In [2]:
"""Update income data per Tom Luben's suggestion in technical review, quartile-ranked within CBSAs where available or state if not.

From Tom Luben's 6/14/2024 email:

•	For all tracts within a CBSA: Identify if census tract is in 1st, 2nd, 3rd, or 4th quartile of Median Household Income compared with all other census tracts in that CBSA.
•	For all tracts outside of a CBSA: Identify if census tract is in 1st, 2nd, 3rd, or 4th quartile of Median Household Income compared with all other census tracts in the same state that are not in a CBSA.

Data dictionary:

CBSAFP10: 2010 Census Core-Based Statistical Area (CBSA) unique five-digit identifiers. Rows with blanks indicate a census tract is outside of a CBSA.
HINC0A_10: 2008-2012 American Community Survey (ACS) sample median household income estimated within 2010 tract boundaries.

"""

import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(
    "C:/Users/RRice/OneDrive - Environmental Protection Agency (EPA)/exposure disparities/luben income data/GentrificationIndex_Tracts10_export/GentrificationIndex_Tracts10_export.csv",
    dtype={"GEOID10": "str", "HINC0A_10": "float"},
)

In [4]:
# fill placeholder values with NA in 2010 household income column
# turn ID fields into str
# appropriately handle missing values
df["HINC0A_10"] = df["HINC0A_10"].replace(-999, np.nan)

df["CBSAFP10"] = df["CBSAFP10"].astype(str).str.replace(".0", "")
df["CBSAFP10"] = df["CBSAFP10"].replace("nan", pd.NA)

df.set_index("GEOID10", inplace=True)

# 618 NA values, 0.86% of the data. Dropping these rows
df = df.dropna(subset=["HINC0A_10"])

In [ ]:
df["HINC0A_10"]

In [ ]:
def quartile_rank(df):
    """Quartile rank 2010 household income."""
    return pd.qcut(df["HINC0A_10"], 4, duplicates="drop", labels=False) + 1


# where there is a CBSAFP10 value, group tracts by CBSA and quartile rank
df_cbsa = df.loc[df.CBSAFP10.notna()].copy()

# group tracts outside of CBSAs by state and quartile rank
df_no_cbsa = df.loc[df.CBSAFP10.isna()].copy()

df_cbsa_income = df_cbsa.groupby("CBSAFP10").apply(quartile_rank).reset_index()
df_state_income = df_no_cbsa.groupby("STATE").apply(quartile_rank).reset_index()

outdf = pd.concat([df_cbsa_income, df_state_income])[["GEOID10", "HINC0A_10"]]

In [ ]:
# there are 36 duplicates due to values falling on the border between quartiles
outdf.loc[outdf.GEOID10.duplicated(keep="first")].shape[0]

In [12]:
# randomly assign one of the quartiles to the duplicates
outdf = outdf.sample(frac=1).drop_duplicates(subset=["GEOID10"], keep="first")

In [ ]:
# there are now 0 duplicates
outdf.loc[outdf.GEOID10.duplicated(keep="first")].shape[0]

In [14]:
outdf.columns = ["GEOID10", "Income quartile"]

In [15]:
outdf.set_index("GEOID10").to_csv(
    "C:/Users/RRice/OneDrive - Environmental Protection Agency (EPA)/exposure disparities/thesis intermediate dataset/updated Census tract income quartiles 6-17-2024.csv"
)

In [ ]:
outdf